# Data

The main data I will use in this project comes from YouTube which is my watch history data. It was sent by YouTube per my request in html format. Since I intend to use pandas and plotly for data visualasation purposes the first order of business will be to clean up the html data and conver it to csv format.

# Ads

The html file containing the data related to my watch history also included the ads I watche. Below I clean the data of those ads which there were about 9.5k ads among the 55k total videos I have watched.

In [1]:
from pathlib import Path
from bs4 import BeautifulSoup
import lxml

# Define the path to the HTML file
data_file = Path("Data") / "izleme geçmişi.html"

# Check if the file exists
if data_file.exists():
    print(f"File found: {data_file}")
else:
    print("File not found!")

# Open and parse the file
with open(data_file, "r", encoding="utf-8") as file:
    soup = BeautifulSoup(file, "lxml")

ads = soup.find_all(string=lambda text: text and "Google Reklamlar" in text.replace("\xa0", " ").strip())
print(f"Found {len(ads)} ads")  # Debugging: Output the number of matches

for ad in ads:
    ad_div = ad.find_parent("div", class_=lambda classes: classes and "outer-cell" in classes and "mdl-cell--12-col" in classes)
    if ad_div:
        ad_div.decompose()


# Save the cleaned chunk
cleaned_data_path = Path("Data") / "addfree_data.html"
with open(cleaned_data_path, "w", encoding="utf-8") as file:
    file.write(soup.prettify())

print(f"Cleaned HTML saved to {cleaned_data_path}")

File found: Data\izleme geçmişi.html
Found 9568 ads
Cleaned HTML saved to Data\addfree_data.html


In [ ]:
from pathlib import Path
from bs4 import BeautifulSoup
import lxml

# Define the path to the cleaned HTML file
data_file = Path("Data") / "cleaned_chunk.html"

# Open and parse the file
with open(data_file, "r", encoding="utf-8") as file:
    soup = BeautifulSoup(file, "lxml")  # Using lxml for faster parsing

# Find all strings containing "yayını görüntülendi"
posts = soup.find_all(string=lambda text: text and "yayını görüntülendi" in text)

print(f"Found {len(posts)} community post records.")  # Debugging: Output the number of matches

# Remove parent divs containing these records
for post in posts:
    post_div = post.find_parent("div", class_=lambda classes: classes and "outer-cell" in classes and "mdl-cell--12-col" in classes)
    if post_div:
        print(f"Removing community post div: {post_div}")  # Debugging: Confirm removal
        post_div.decompose()

# Save the cleaned HTML
cleaned_file_path = Path("Data") / "ncp_chunk.html"
with open(cleaned_file_path, "w", encoding="utf-8") as file:
    file.write(soup.prettify())

print(f"Cleaned HTML saved to {cleaned_file_path}")
